In [17]:
import torch
from torch import nn
import numpy as np

from torch.utils.data import Dataset, DataLoader

In [8]:
class ResidualBlock(nn.Module):
    
    def __init__(self,in_channels,out_channels,stride=1,kernel_size=3,padding=1,bias=False):
        super(ResidualBlock,self).__init__()
        
        self.cnn1 = nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(True)
        )
        self.cnn2 = nn.Sequential(
            nn.Conv2d(out_channels,out_channels,kernel_size,1,padding,bias=False),
            nn.BatchNorm2d(out_channels)
        )
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride,bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Sequential()
            
    def forward(self,x):
        residual = x
        x = self.cnn1(x)
        x = self.cnn2(x)
        x += self.shortcut(residual)
        x = nn.ReLU(True)(x)
        return x

class ResNet34(nn.Module):
    
    def __init__(self):
        super(ResNet34, self).__init__()
        
        self.layers = [
            nn.Sequential(
                nn.Conv2d(1, 64, kernel_size=2, stride=2, padding=3, bias=False),
                nn.BatchNorm2d(64),
                nn.ReLU(True)
            ),
            nn.Sequential(
                nn.MaxPool2d(1, 1),
                ResidualBlock(64, 64),
                ResidualBlock(64, 64, 2)
            ),
            nn.Sequential(
                ResidualBlock(64, 128),
                ResidualBlock(128, 128, 2)
            ),
            nn.Sequential(
                ResidualBlock(128, 256),
                ResidualBlock(256, 256, 2)
            ),
            nn.Sequential(
                ResidualBlock(256, 512),
                ResidualBlock(512, 512, 2)
            ),
            self.view,
            nn.AvgPool2d(2),
            nn.Linear(512, 11),
            nn.Linear(512, 168),
            nn.Linear(512, 7),
        ]
        
    def view(x):
        return x.view(x.size(0), -1)
    
    def forward(self,x):
        for l in self.layers:
            x = l(x)
        return x

In [18]:
class GraphemeDataset(Dataset):
    
    def __init__(self,df,label,_type='train'):
        self.df = df
        self.label = label
        
    def __len__(self):
        return len(self.df)
        
    def __getitem__(self,idx):
        label1 = self.label.vowel_diacritic.values[idx]
        label2 = self.label.grapheme_root.values[idx]
        label3 = self.label.consonant_diacritic.values[idx]
        image = self.df.iloc[idx][1:].values.reshape(64,64).astype(np.float)
        return image,label1,label2,label3

In [19]:
model = ResNet34()
optimizer = torch.optim.Adam(model.parameters(), lr=4e-4)
criterion = nn.CrossEntropyLoss()
batch_size = 32

ValueError: optimizer got an empty parameter list

In [ ]:
epochs = 50 # original 50
model.train()
losses = []
accs = []
for epoch in range(epochs):
    reduced_index =train.groupby(['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']).apply(lambda x: x.sample(5)).image_id.values
    reduced_train = train.loc[train.image_id.isin(reduced_index)]
    reduced_data = data_full.loc[data_full.image_id.isin(reduced_index)]
    train_image = GraphemeDataset(reduced_data,reduced_train)
    train_loader = torch.utils.data.DataLoader(train_image,batch_size=batch_size,shuffle=True)
    
    print('epochs {}/{} '.format(epoch+1,epochs))
    running_loss = 0.0
    running_acc = 0.0
    for idx, (inputs,labels1,labels2,labels3) in tqdm(enumerate(train_loader),total=len(train_loader)):
        inputs = inputs.to(device)
        labels1 = labels1.to(device)
        labels2 = labels2.to(device)
        labels3 = labels3.to(device)
        
        optimizer.zero_grad()
        outputs1,outputs2,outputs3 = model(inputs.unsqueeze(1).float())
        loss1 = criterion(outputs1,labels1)
        loss2 = criterion(outputs2,labels2)
        loss3 = criterion(outputs3,labels3)
        running_loss += loss1+loss2+loss3
        running_acc += (outputs1.argmax(1)==labels1).float().mean()
        running_acc += (outputs2.argmax(1)==labels2).float().mean()
        running_acc += (outputs3.argmax(1)==labels3).float().mean()
        (loss1+loss2+loss3).backward()
        optimizer.step()
    #scheduler.step()
    losses.append(running_loss/len(train_loader))
    accs.append(running_acc/(len(train_loader)*3))
    print('acc : {:.2f}%'.format(running_acc/(len(train_loader)*3)))
    print('loss : {:.4f}'.format(running_loss/len(train_loader)))
torch.save(model.state_dict(), 'resnet34_50epochs_saved_weights.pth')